In [ ]:
from google.colab import files
uploaded = files.upload()



Saving dataset.csv to dataset.csv


In [ ]:
import pandas as pd

df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity"]].to_csv("parking_stream_model1.csv", index=False)


In [ ]:
!pip install pathway --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pathway as pw

class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int


In [ ]:
data = pw.demo.replay_csv("parking_stream_model1.csv", schema=ParkingSchema, input_rate=1000)

data = data.with_columns(
    t = data.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)


In [ ]:
BASE_PRICE = 10.0
ALPHA = 5.0
MIN_PRICE = 5.0
MAX_PRICE = 20.0

model1_output = data.with_columns(
    price = pw.apply(
        lambda occ, cap: max(MIN_PRICE, min(MAX_PRICE, BASE_PRICE + ALPHA * (occ / cap))),
        data.Occupancy, data.Capacity
    )
)


In [ ]:
# Check the latest computed prices
model1_output.select(
    timestamp = model1_output.Timestamp,
    lot_id = model1_output.SystemCodeNumber,
    occupancy = model1_output.Occupancy,
    capacity = model1_output.Capacity,
    price = model1_output.price
).show()


Column
    [0] Row
        [0] Markdown(str)
        [1] TooltipIcon(value='Table depends o...)
    [1] Tabulator(disabled=True, height=400, page_size=10, pagination='local', show_index=False, sizing_mode='stretch_width', value=Empty DataFrame
Columns: [...)

In [ ]:
pw.io.csv.write(
    model1_output.select(
        timestamp = model1_output.Timestamp,
        lot_id = model1_output.SystemCodeNumber,
        occupancy = model1_output.Occupancy,
        capacity = model1_output.Capacity,
        price = model1_output.price
    ),
    "model1_prices.csv"  # Path to output CSV
)

# Run the pipeline
pw.run()


Output()

In [ ]:
import pandas as pd

model1_df = pd.read_csv("model1_prices.csv")
model1_df['timestamp'] = pd.to_datetime(model1_df['timestamp'])
model1_df = model1_df.sort_values('timestamp')
model1_df.head()


,timestamp,lot_id,occupancy,capacity,price,time,diff
0,2016-10-04 07:59:00,BHMNCPHST01,237,1200,10.987500,1751741486090,1
80,2016-10-04 07:59:00,Others-CCCPS119a,195,2803,10.347842,1751741486770,1
66,2016-10-04 07:59:00,Others-CCCPS98,588,3103,10.947470,1751741486770,1
58,2016-10-04 07:59:00,BHMBCCTHL01,120,387,11.550388,1751741486770,1
44,2016-10-04 07:59:00,Others-CCCPS135a,1081,3883,11.391965,1751741486770,1


In [ ]:
import pandas as pd

# Load exported prices from CSV
model1_df = pd.read_csv("model1_prices.csv")

# Convert timestamp to datetime
model1_df['timestamp'] = pd.to_datetime(model1_df['timestamp'])
model1_df = model1_df.sort_values('timestamp')


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

output_notebook()

# Pick a parking lot
lot_id = model1_df['lot_id'].iloc[0]
lot_data = model1_df[model1_df['lot_id'] == lot_id]

# Bokeh data source
source = ColumnDataSource(lot_data)

# Create the figure
p = figure(
    x_axis_type="datetime",
    title=f"Model 1 Price Over Time – Lot {lot_id}",
    x_axis_label="Time",
    y_axis_label="Price ($)",
    width=800,
    height=350
)

# Plot line + points
p.line(x='timestamp', y='price', source=source, line_width=2, color="navy", legend_label="Price")
p.circle(x='timestamp', y='price', source=source, size=5, color="orange")

# Layout polish
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.toolbar.logo = None
p.toolbar.autohide = True

show(p)


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

output_notebook()

import pandas as pd

# Load final CSV
df = pd.read_csv("model1_prices.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

# Get unique parking lots
lots = df['lot_id'].unique()

# Create plots per lot
plots = []
palette = Category10[10]

for i, lot in enumerate(lots[:10]):  # support up to 10 lots
    lot_df = df[df['lot_id'] == lot]
    source = ColumnDataSource(lot_df)

    p = figure(
        x_axis_type="datetime",
        title=f"Real-Time Pricing: Lot {lot}",
        x_axis_label="Time",
        y_axis_label="Price ($)",
        width=800,
        height=300
    )

    color = palette[i % len(palette)]
    p.line(x='timestamp', y='price', source=source, color=color, line_width=2, legend_label=f"Lot {lot}")
    p.circle(x='timestamp', y='price', source=source, color=color, size=5)
    p.add_tools(HoverTool(tooltips=[("Time", "@timestamp{%F %T}"), ("Price", "@price")],
                          formatters={"@timestamp": "datetime"}))
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    plots.append(p)

# Show all plots in a column
show(column(*plots))
